In [293]:
import requests
import csv
import os
import urllib
import sys
import importlib
import pandas as pd
from shapely.geometry import Point, Polygon
import time
import datetime
from datetime import datetime
from time import sleep

sys.path.insert(1, '../utils')

import utils

In [317]:
s = requests.Session()

In [294]:
importlib.reload(utils)

<module 'utils' from '../utils/utils.py'>

In [295]:
region = 'New York'
extent = 'core'
county_ids = utils.county_ids.get_county_ids(region, extent)
input_date = '2020-05-01'
xmin, xmax, ymin, ymax = utils.geometry.osm_bounds(region, county_ids, extent, file = False, raw = True)

url = 'https://api.transitfeeds.com'

In [296]:
# bounding box

coords = [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]
poly = Polygon(coords)

In [319]:
# api key

key = '9efe3e4d-9b04-467c-9c72-0cc3cf666fd0'
#key = 'f2a91a7e-154d-434a-8083-2cd18e25f3d2'

In [320]:
output_feed_info = [list(['operator_name', 'operator_website', 'operator_location', 'operator_metro', 'feed_id', 'date_query', 
                         'date_fetched', 'earliest_calendar_date', 'latest_calendar_date', 'transit_feeds_url'])]

In [321]:
# gets all available locations on transit feeds

response = s.get(
    url+'/v1/getLocations',
    params = {'key': key
       
    }
)
locations = response.json()['results']['locations']
sleep(15)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [300]:
# filters locations to only those within bounding box

location_ids = []
for operator in locations:
    pt = Point(operator['lng'], operator['lat'])
    if pt.within(poly) is True: 
        location_ids.append(operator['id'])
location_ids

[116, 119, 294, 91, 118, 117]

In [309]:
# loops through all locations

for ids in location_ids:
    print(ids)
    response = requests.get(
        url+'/v1/getFeeds',
        params = {'key': key, 'location' : ids, 'type': 'gtfs', 'limit': 200

        }
    )
    feeds = response.json()['results']['feeds']
    sleep(15)
    
    for row in feeds:
        print(row['id'])
        response = requests.get(
            url+'/v1/getFeedVersions',
            params = {'key': key, 'feed' : row['id'], 'limit' : 999

            }
        )
        feed_info = response.json()['results']['versions']
        sleep(15)
        
        for version in feed_info:

        # checks latest version to input date

            if input_ts > version['ts']:

                # writes table for metadata

                op_name = version['f']['t']
                op_url = version['f']['u']['i']
                op_loc = version['f']['l']['t']
                op_id = version['f']['id']
                date_fetched = str(datetime.utcfromtimestamp(version['ts']).strftime('%Y-%m-%d'))
                earliest_calendar_date = str(datetime.strptime(version['d']['s'], '%Y%m%d').date())
                latest_calendar_date =  str(datetime.strptime(version['d']['f'], '%Y%m%d').date()) 
                url = version['url']
                temp = list([op_name, op_url, op_loc, region, op_id, input_date, date_fetched, earliest_calendar_date,
                           latest_calendar_date, url])
                output_feed_info.append(temp)

                # downloads gtfs

                urllib.request.urlretrieve(feed_info[1]['url'], "../gtfs/feeds_" + input_date + "/" + op_name + '-' + input_date + ".zip")

                break
    

116


JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [303]:
output_feed_info

[['operator_name',
  'operator_website',
  'operator_location',
  'operator_metro',
  'feed_id',
  'date_query',
  'date_fetched',
  'earliest_calendar_date',
  'latest_calendar_date',
  'transit_feeds_url'],
 ['MTA Bronx GTFS',
  'http://mta.info/developers/download.html',
  'Bronx, NY, USA',
  'New York',
  'mta/81',
  '2020-05-01',
  '2020-04-04',
  '2020-01-18',
  '2020-05-02',
  'https://transitfeeds.com/p/mta/81/20200403/download']]